In [ ]:
##### 1-1 데이터

# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제프로젝트 AI hub 기타 등등
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [ ]:
# 텍스트 전처리
# unicode 정규화, 특수문자처리
# NFD : Normalization form Decomposition --> 분해가능한 모든 문자를 분해한다
import unicodedata
unicodedata.normalize('NFD', "e'") 

"e'"

<span style="font-size:12px">

- $\acute{e}$  --->  단일문자 U+00E9
- $e$ + ' 
- $e$ + 악센트 
- unicode 정규화 : 같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 갖는 경우 --> 동일한 내부 표현으로 바꿔주는 과정 


In [ ]:
##### 1-2 텍스트 전처리
### 전처리 함수
def preprocess_stentence(sentence, is_korean = False):
    '''
    Unicode 정규화, 특수문자처리
    Arg : 
        sentence : 원본
        is_korean : 한국어 여부
    return : 
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD', sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip() #문자열 시작, 끝 부분, 줄바꿈 문자(\n), 탭(\t) 제거
    # 정규식으로 특수문자 전후에 공백추가
    # 안녕하세요! --> 안녕하세요 !
    # r"([? ! , .])" # --> 문자중에 ? ! , . 나오면 문자를 그룹으로 캡쳐
    # r"\1" 캡쳐한 문자(\1) 앞뒤로 공백을 하나씩 넣는다
    sentence = re.sub(r"([? ! , .])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료 토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

In [33]:
##### 1-2 텍스트 전처리 적용
kor_processed = [ preprocess_stentence(kor) for kor in korean_sentences]  
eng_processed =[ preprocess_stentence(eng) for eng in english_sentences]

print (f'한국어 전처리 결과 : {kor_processed[0]} \n영어 전처리 결과 : {eng_processed[0]}')

한국어 전처리 결과 : <start>안녕하세요<end> 
영어 전처리 결과 : <start>hello<end>


In [16]:
##### 1-3 토크나이저
# 문장을 정수 시퀀스로 변환
def create_tokenizer (sentence):
    '''
    단어를 정수 인덱스로 변환
    vocabulary 구축, word to index mapping
    Args:
        sentence : 문장리스트
    returns:
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', #필터 비활성화
        oov_token='<unk>'
    )
    # 모든 문장으로 단어 사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer


In [35]:
##### 1-3 토크나이저 적용
# 한국어 영어 각각의 토크나이저 생성
kor_tokenizer = create_tokenizer(kor_processed)
eng_tokenizer = create_tokenizer(eng_processed)
kor_tokenizer
eng_tokenizer

# 단어 사전 크기 확인
kor_vocab_size = len(kor_tokenizer.word_index)+1
eng_vocab_size = len(eng_tokenizer.word_index)+1
print(f'한국어 사전 크기 : {kor_vocab_size} \n영어 사전 크기 : {eng_vocab_size}')

한국어 사전 크기 : 25 
영어 사전 크기 : 30


In [ ]:
#### 1-4 정수 시퀀스 변경 (encode)
def encode_sentence(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 무장을 동일 길이로 맞추는
    Arg:
        tokenizer 
        sentence : 문장리스트
        maxlen : 가장 긴 문장
    returns:
        패딩된 점수 시퀀스 배열 tf의 pad_sequence
    '''
    sequence = tokenizer.texts_to_sequences(sentences) # 예) [i love you] --> [1,5,7]

    # 길이 맞추기 (패딩 추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
            sequence,
            maxlen = maxlen,
            padding = 'post'
    )
    return padded

<span style="font-size:12px">

#### 패딩방식
#### 🔹 RNN/LSTM/GRU vs Transformer 패딩 방식 정리
##### ✅ 1) RNN / LSTM / GRU — `padding='pre'`
- RNN 계열은 **왼쪽 → 오른쪽으로 순차적으로 읽는 구조**.
- 마지막 시점의 hidden state(`h_last`)가 **전체 문장을 요약하는 핵심 정보**가 됨.
- 그래서 **진짜 단어들이 뒤쪽에 위치하도록** 앞쪽에 패딩을 넣는 `pre-padding`을 사용.
  - 예: `[PAD, PAD, 단어1, 단어2, 단어3]`
- 이렇게 하면 RNN이 **마지막에 읽는 부분이 실제 단어**가 되어  
  `h_last`가 문맥 정보를 정확히 담을 수 있음.


##### ✅ 2) Transformer — `padding='post'`
- Transformer는 RNN처럼 순차적으로 읽지 않고,  
  **전체 단어를 동시에 보면서 Attention을 계산**.
- 문맥 요약이 "마지막 단어"에 의존하지 않음.
- 위치 정보는 **positional encoding**으로 해결되므로  
  문장의 자연스러운 순서를 유지하는 `post-padding`이 적합.
  - 예: `[단어1, 단어2, 단어3, PAD, PAD]`
- 뒤쪽 패딩은 self-attention 계산 시 **masking으로 처리되어 무시됨**.


##### 🎯 한줄 요약
- **RNN 계열 → 마지막 hidden state가 중요 → `pre-padding`**  
- **Transformer → 순차 처리 아님 + masking 가능 → `post-padding`**



| 모델               | 패딩 방식            | 이유                                                               |
| ---------------- | ---------------- | ---------------------------------------------------------------- |
| **RNN/LSTM/GRU** | **pre-padding**  | 중요한 단어가 뒤쪽에 오도록 해서 **마지막 hidden state**가 문맥 요약을 잘 갖도록            |
| **Transformer**  | **post-padding** | 순차 처리 없음. 위치 정보는 positional embedding으로 해결되므로 앞/뒤 순서만 그대로 유지하면 됨 |



In [39]:
# 최대 시퀀스의 길이 결정(가장 긴 문장기준)
max_kor_len = max( len(s.split()) for s in kor_processed)
max_eng_len = max( len(s.split()) for s in eng_processed)
print(f'한국어 시퀀스의 최대 길이 : {max_eng_len} \n영어 시퀀스의 최대 길이 : {max_eng_len}')
print('====================================')


##### 1-4 정수 시퀀스 변경 (encode) 인코딩 수행
kor_tensor = encode_sentence(kor_tokenizer, kor_processed, max_kor_len)
eng_tensor = encode_sentence(eng_tokenizer, eng_processed, max_eng_len)
print (f'한국어 전처리 결과 : {kor_processed[0]} \n영어 전처리 결과 : {eng_processed[0]}')
print(f'한국어 인코딩 첫번째 문장 : {kor_tensor[0]} \n영어 인코딩 첫번째 문장: {eng_tensor[0]}')
print('====================================')


# tensorflow Dataset 객체
# 배치처리 와 셔플
BUFFER_SIZE = len(kor_tensor)
BATCH_SIZE = 2

dataset = tf.data.Dataset.from_tensor_slices( (kor_tensor, eng_tensor))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print('데이터셋 준비완료')
print(f'전체 샘플수 : {len(kor_tensor)}')
print(f'배치 크기 : {BATCH_SIZE}')
print(f'배치 수 : {len(kor_tensor) // BATCH_SIZE}')


한국어 시퀀스의 최대 길이 : 5 
영어 시퀀스의 최대 길이 : 5
한국어 전처리 결과 : <start>안녕하세요<end> 
영어 전처리 결과 : <start>hello<end>
한국어 인코딩 첫번째 문장 : [3 0 0] 
영어 인코딩 첫번째 문장: [7 0 0 0 0]
데이터셋 준비완료
전체 샘플수 : 10
배치 크기 : 2
배치 수 : 5


<span style="font-size:12px">


----------------- step2 -------------------------------------

- seq2seq 구조
- Encoder
    - 입력문장 --> Embedding --> LSTM + Hidden States
- Decoder
    - <start> 토큰 --> Embedding --> LSTM + context --> 단어예측
<br>    예측단어 --> 다음입력 --> 반복(<end> 나올때까지)


In [ ]:
# Encoder 클래스
class Encoder(tf.keras.Model):
    '''
    입력문장을 고차원 벡터로 압축
    Embedding --> LSTM --> Hidden States 출력
    구조 : 
        입력(정수시퀀스)  --> Embedding --> LSTM --> 모든 타임스텝의 출력
    '''
    def __init__ (self, vocab_size, embedding_dim, enc_units, batch_size):
        '''
        Arg:
            vocab_size : 단어사전 크기(임베딩 테이블 크기)
            embedding_dim : 임베딩 벡터 차원 (방향이 있는 벡터로 만듬)
            enc_units : LMS 유닉 수 (hidden state 차원)
            batch_size : 배치크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units

        # Embedding Layer : 정수 --> 밀집벡터(Denserf Vector) 
        # 작원 차원 대부분의 값이 0이 아님. 연속된 실수값 공간 효율성
        # 학습 과정을 통해 단어간 의미적 관계 학습 --> 유사 단어는 가까운 벡터 공간에 위치
        # 단점: 훈련필요(사전학습 또는 임베딩을 학습)

        # 희소벡터 : 대부분의 값이 0 (대표적으로 원-핫벡터) 차원이 크다/ 단어간 충돌이 없다/단어간 유사도표현 ㅇ못함 
        # 밀집벡터 : 학습기반 (Word2Vec Glove Embedding)
        # 희소벡터 : 규칙기간 (원핫 Bow) --> 초창기 자연어 모델

        # mask_zero = True 패딩에 마스크처리를 해서 모델이 해석하지 않게 한다
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, embedding_dim, mask_zero = True
        )
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequence = True, #[batch, seq_len, units]
            return_state =True, #(output, h, c)
            recurrent_initializer = 'glorot_uniform' #가중치 초기화
        )
    def call (self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args:
            x: 입력 시퀀스 [ batch_size, seq_len ]
            hidden: 초기 hidden state (첫 호출시 0 벡터)
        Returns :
            output: 모든타임스템프의 출력  [batch, seq_len, enc_units]
            state_h : 마지막 hidden state [batch, enc_units]
            state_c : 마지막 cell state [batch, enc_units]
        '''

        # 1단계 emb
        x= self.embedding(x)
        output,  state_h, state_c = self.lstm(x,initial_state=hidden)
        return output, state_h, state_c

In [ ]:
class ABC():
    def __call__(self, data):
        self.call(data)

    def call(self,data):
        print('호출', data)

a= ABC()
a(100)

호출 100


-